Topic Modeling with Gensim

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#17howtofindtheoptimalnumberoftopicsforlda

In [3]:
import glob
import random
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim import corpora, models
from tqdm import tqdm
import numpy as np

stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in simple_preprocess(text, min_len = 4):
        if token not in STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

cat_list = glob.glob ("cuisines/*")
cat_size = len(cat_list)

random.seed(0)
cat_names = []
cat_text = []
# sample_size = min(30, cat_size)
# cat_sample = sorted(random.sample(range(cat_size), sample_size))
cat_sample = range(0, cat_size)

count = 0
for i in cat_sample:
    cat_names.append(cat_list[i].replace("\\", "/").split('/')[-1][:-4].replace("_"," "))
    with open(cat_list[i]) as f:
        cat_text.append(f.read().replace("\n", "").replace("\r",""))

processed_docs = [preprocess(text) for text in tqdm(cat_text)]
dictionary = corpora.Dictionary(processed_docs)
print("Before prunn:%d"%(len(dictionary)))
dictionary.filter_extremes(no_below = 2, no_above = 0.5)
print("After prunn:%d"%(len(dictionary)))
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

100%|██████████| 147/147 [30:21<00:00, 12.39s/it] 


Before prunn:193047
After prunn:100000


In [163]:
import math
import sklearn
from scipy import spatial

def cosine_similarity(a, b):
    b = dict(b)
    norm_a = 0
    norm_b = 0
    denom = 0
    for a_i, a_v in a:
        norm_a += a_v * a_v
        if a_i in b:
            denom += a_v * b[a_i]
    for b_i in b:
        norm_b += b[b_i] * b[b_i]
    
    norm_a = math.sqrt(norm_a)
    norm_b = math.sqrt(norm_b)
    
    return denom / (norm_a * norm_b)

def top_n(df, n, thresh_hold = 0.1):
    df_count = np.zeros(df.shape)
    df_bak = df
    df_count[df >= thresh_hold] = 1
    _counts = np.sum(df_count, axis=1)
    max_index = []
    for i in range(0, n):
        _index = np.argmax(_counts)
        max_index.append(_index)
        _counts[_index] = -1
    
    return df.iloc[max_index][df.columns[max_index]]

def slice_df_by_name(df,names):
    return df.loc[names][names]

def format_obj(df, groups):
    _nodes = "nodes"
    _links = "links"
    json_obj = {_nodes:[], _links:[]}
    sorted_names = []
    name2gid = dict()
    for g in range(0, len(groups)):
        for name in groups[g]:
            name2gid[name] = g
            if name in df.columns:
                sorted_names.append(name)
    
    df = slice_df_by_name(df, sorted_names)
    for c_name in df.columns:
        json_obj[_nodes].append({"name": c_name, "group":name2gid[c_name]})
    
    for i in range(0, df.shape[0] - 1):
        for j in range(i + 1, df.shape[0]):
            json_obj[_links].append({"source":i, "target":j, "value":float(df.iloc[i][j])})
    
    return json_obj

def corpus2matrix(corpus, vector_dimension):
    _corpus_matrix = np.zeros([len(corpus), vector_dimension])
    for i, row in enumerate(corpus):
        for j, v in row:
            _corpus_matrix[i][j] = v
    
    return _corpus_matrix
    
def corpus_similarity(corpus, vector_dimension, distance_func = sklearn.metrics.pairwise.cosine_similarity):
    _corpus_matrix = corpus2matrix(corpus, vector_dimension)
    #Normailzation
#     _corpus_matrix = Normalizer().transform(_corpus_matrix)    
    return distance_func(_corpus_matrix)


def corpus_similarity_1(corpus):
    _sim = np.zeros([len(corpus), len(corpus)])

    for i in tqdm(range(0, len(corpus) - 1)):
        _sim[i][i] = 1
        for j in range(i + 1, len(corpus)):
            _sim[i][j] = cosine_similarity(corpus[i], corpus[j])
            _sim[j][i] = _sim[i][j]
    
    return _sim

In [164]:
from sklearn.cluster import KMeans, Birch, DBSCAN
from sklearn.preprocessing import Normalizer
from sklearn import metrics

def kmean_predict(X, n_clusters):
    return KMeans(n_clusters = n_clusters).fit_predict(X)

def birch_predict(X, n_clusters):
    return Birch(compute_labels = True, n_clusters = n_clusters).fit_predict(X)

def dbscan_predict(X, n_clusters):
    return DBSCAN(eps=n_clusters * 0.01, min_samples=1).fit_predict(X)

cluster_method = {"kmean": kmean_predict,
                 "birch": birch_predict,
                 "dbscan": dbscan_predict}

def get_cluster(features_list, feature_dimension, names, num_cluster = -1, method = "kmean", verbose = False):
    X = corpus2matrix(features_list, feature_dimension)
    Norm_X = Normalizer().transform(X)
    
    if num_cluster < 0:
        best_score = -1
        best_k = -1
        for k in range(2, 100):
            y_pred = cluster_method[method](Norm_X, k)
            _score = metrics.silhouette_score(Norm_X, y_pred, metric='euclidean')
            if verbose:
                print(_score)
            if _score > best_score:
                best_k = k
                best_score = _score
        if verbose:
            print("Best k:%d"%(best_k))
    else:
        best_k = num_cluster
        
    y_pred = cluster_method[method](Norm_X, best_k)
    clusters = dict()
    name2cluster = dict()
    for i in range(0, len(y_pred)):
        name2cluster[names[i]] = y_pred[i]
        if y_pred[i] in clusters:
            clusters[y_pred[i]].append(names[i])
        else:
            clusters[y_pred[i]] = [names[i]]

    return (clusters, name2cluster)

In [165]:
import numpy as np
import pandas as pd
import json
        
sim = corpus_similarity(corpus, len(dictionary))
sim_clusters, i = get_cluster(corpus, len(dictionary), cat_names, 10, method='birch')

sim_df = pd.DataFrame(sim)
sim_df.index = cat_names
sim_df.columns = cat_names

sim_data = top_n(sim_df, 50)
selected_names = data.columns

with open("display/output.json", "w") as f:
    f.write(json.dumps(format_obj(sim_data, sim_clusters)))

In [6]:
# import seaborn as sns; 
# import matplotlib.pyplot as plt

# sample = 20
# ax = sns.heatmap(data.iloc[0:sample][data.columns[0:sample]],cmap="YlGnBu", xticklabels=True, yticklabels=True)
# plt.show()

In [166]:
from gensim.models import TfidfModel
import json

tfidf_model = TfidfModel(corpus)
tfidf_corpus = tfidf_model[corpus]

tfidf_sim = corpus_similarity(tfidf_corpus, len(dictionary))
tfidf_sim_clusters, i = get_cluster(tfidf_corpus, len(dictionary), cat_names, 10, method='birch')

tfidf_sim_df = pd.DataFrame(tfidf_sim)
tfidf_sim_df.index = cat_names
tfidf_sim_df.columns = cat_names
tfidf_data = top_n(tfidf_sim_df, 50)
#tfidf_data = slice_df_by_name(tfidf_sim_df, selected_names)
                              
with open("display/tfidf_output.json", "w") as f:
    f.write(json.dumps(format_obj(tfidf_data, tfidf_sim_clusters)))

In [171]:
get_cluster(tfidf_corpus, len(dictionary), cat_names, 10, method='birch')

({1: ['Filipino',
   'Indonesian',
   'Ethnic Food',
   'Shanghainese',
   'Health Markets',
   'Malaysian',
   'Chinese',
   'Dim Sum',
   'Food Trucks',
   'Singaporean',
   'Cheese Shops',
   'Taiwanese',
   'Szechuan',
   'Cantonese',
   'Shaved Ice',
   'Asian Fusion',
   'Do-It-Yourself Food',
   'Hawaiian',
   'Mongolian',
   'Karaoke',
   'Specialty Food',
   'Meat Shops',
   'Street Vendors',
   'Food Stands',
   'Fruits & Veggies',
   'Barbeque',
   'Korean'],
  0: ['Donuts',
   'German',
   'Sushi Bars',
   'Japanese',
   'Latin American',
   'Cuban',
   'Creperies',
   'Salvadoran',
   'Venezuelan',
   'Fondue',
   'French',
   'Arcades',
   'Argentine',
   'Bubble Tea',
   'Steakhouses',
   'Cajun-Creole',
   'Soul Food',
   'Brazilian',
   'Hot Pot',
   'Caribbean',
   'Southern',
   'Ramen',
   'Peruvian'],
  2: ['Chicken Wings',
   'Pool Halls',
   'American (New)',
   'Sandwiches',
   'Desserts',
   'Lounges',
   'Sports Bars',
   'Comfort Food',
   'Cafes',
   'Fast F

In [172]:
get_cluster(corpus, len(dictionary), cat_names, 10, method='birch')

({1: ['Filipino',
   'Indonesian',
   'Ethnic Food',
   'Shanghainese',
   'Health Markets',
   'Malaysian',
   'Chinese',
   'Dim Sum',
   'Food Trucks',
   'Singaporean',
   'Cheese Shops',
   'Taiwanese',
   'Szechuan',
   'Cantonese',
   'Shaved Ice',
   'Asian Fusion',
   'Do-It-Yourself Food',
   'Hawaiian',
   'Mongolian',
   'Karaoke',
   'Specialty Food',
   'Meat Shops',
   'Street Vendors',
   'Food Stands',
   'Fruits & Veggies',
   'Barbeque',
   'Korean'],
  0: ['Donuts',
   'German',
   'Sushi Bars',
   'Japanese',
   'Latin American',
   'Cuban',
   'Creperies',
   'Salvadoran',
   'Venezuelan',
   'Fondue',
   'French',
   'Arcades',
   'Argentine',
   'Bubble Tea',
   'Steakhouses',
   'Cajun-Creole',
   'Soul Food',
   'Brazilian',
   'Hot Pot',
   'Caribbean',
   'Southern',
   'Ramen',
   'Peruvian'],
  2: ['Chicken Wings',
   'Pool Halls',
   'American (New)',
   'Sandwiches',
   'Desserts',
   'Lounges',
   'Sports Bars',
   'Comfort Food',
   'Cafes',
   'Fast F

In [178]:
get_cluster(tfidf_corpus, len(dictionary), cat_names, method='birch', verbose = True)

0.017620041329974336
0.01235593098258466
0.008589748461113248
0.0075681250605165755
0.012460989141166855
0.011084981192750004
0.01611131374226938
0.019274990483674016
0.02478196597379302
0.025311125013200392
0.02565099907670778
0.027190926382541253
0.028013906504835647
0.025784179729773316
0.02851685914810193
0.030729134566970368
0.03227300723178682
0.03373334407059898
0.03630136086029685
0.03720323098415931
0.03966775275127193
0.041961392433663106
0.04203317218630401
0.04227922409127466
0.040790006001647776
0.04056511573811791
0.04045669834378565
0.03912931123487523
0.03435154384230499
0.037552585492771746
0.04113476998651711
0.04234293925862348
0.041413830414528334
0.04381405356801352
0.04568338832952778
0.04603618582611435
0.045639373643105184
0.0494673975105572
0.04884068100252001
0.049666913497354435
0.04810798811555406
0.04930450298850116
0.052869910735260275
0.05116441469996828
0.04979748504174534
0.048736190973620516
0.052678043828067726
0.04924073030656186
0.052631262657982285

/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (75). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (76). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (77). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (78). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)


0.045543471547476344
0.045543471547476344
0.045543471547476344
0.045543471547476344


/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (79). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (80). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (81). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (82). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)


0.045543471547476344
0.045543471547476344
0.045543471547476344
0.045543471547476344


...

0.045543471547476344
0.045543471547476344
0.045543471547476344
0.045543471547476344


/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (95). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (96). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (97). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (98). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)


0.045543471547476344
0.045543471547476344
Best k:64


/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/sklearn/cluster/birch.py:629: ConvergenceWarning: Number of subclusters found (74) by Birch is less than (99). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)


({63: ['Filipino', 'Karaoke'],
  41: ['Donuts', 'Creperies', 'Bubble Tea'],
  45: ['Indonesian', 'Asian Fusion'],
  23: ['German'],
  58: ['Sushi Bars', 'Japanese'],
  40: ['Chicken Wings', 'Portuguese'],
  2: ['Basque', 'Tapas Bars', 'Spanish', 'Tapas-Small Plates'],
  35: ['Ethnic Food', 'Hawaiian', 'Specialty Food'],
  31: ['Latin American', 'Salvadoran', 'Venezuelan'],
  56: ['Shanghainese', 'Chinese', 'Szechuan', 'Mongolian'],
  11: ['Health Markets',
   'Cheese Shops',
   'Do-It-Yourself Food',
   'Meat Shops',
   'Fruits & Veggies'],
  30: ['Middle Eastern',
   'Lebanese',
   'Mediterranean',
   'Turkish',
   'Arabian',
   'Persian-Iranian',
   'Kosher',
   'Moroccan',
   'Hookah Bars',
   'Greek'],
  6: ['Pool Halls',
   'Sports Bars',
   'Dive Bars',
   'Cocktail Bars',
   'Bars',
   'Local Flavor'],
  57: ['American (New)'],
  61: ['Malaysian', 'Singaporean', 'Taiwanese'],
  54: ['Cuban'],
  59: ['Sandwiches', 'Cheesesteaks'],
  33: ['Buffets'],
  39: ['Desserts'],
  4: ['Veg

In [53]:
from time import time
from gensim.models.coherencemodel import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

num_topics = 100

t0 = time()
lda_model = models.LdaModel(tfidf_corpus, 
                            num_topics = num_topics, 
                            id2word = dictionary,
                            random_state = 100,
                            eval_every=5, 
                            alpha='auto', 
                            gamma_threshold=0.01)
# lda_model = models.LdaModel(tfidf_corpus, 
#                             num_topics = num_topics, 
#                             id2word = dictionary,
#                             random_state = 100,
#                             update_every = 1,
#                             chunksize = 100,
#                             passes = 10,
#                             alpha = 'auto')

doc_topics = lda_model[tfidf_corpus]
print("Training done in %fs" % (time() - t0))

t0 = time()

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(tfidf_corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model = lda_model, texts = processed_docs, dictionary = dictionary, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print("Evaluation done in %fs" % (time() - t0))

/anaconda3/envs/dm_cap_py3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Training done in 205.340182s

Perplexity:  -27.3974723805885

Coherence Score:  0.42105914279132717
Evaluation done in 537.868681s


In [48]:
#Dosen't work

mallet_path = '../mallet-2.0.8/bin/mallet'

t0 = time()
lda_mallet_model = models.wrappers.LdaMallet(mallet_path, corpus = corpus, num_topics = num_topics, id2word = dictionary)
mallet_doc_topics = lda_mallet_model[corpus]
print("Training done in %fs" % (time() - t0))

# Compute Coherence Score
t0 = time()
coherence_model_ldamallet = CoherenceModel(model = lda_mallet_model, texts = processed_docs, dictionary = dictionary, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)
print("Evaluation done in %fs" % (time() - t0))

Training done in 1167.796796s

Coherence Score:  0.4879352581546337
Evaluation done in 615.806215s


In [169]:
lda_sim = corpus_similarity(doc_topics, num_topics)
lda_sim_clusters, i = get_cluster(doc_topics, num_topics, cat_names, 10, method='birch')

lda_sim_df = pd.DataFrame(lda_sim)
lda_sim_df.index = cat_names
lda_sim_df.columns = cat_names
# lda_data = top_n(lda_sim_df, 50)
lda_data = slice_df_by_name(lda_sim_df, selected_names)

with open("display/lda_output.json", "w") as f:
    f.write(json.dumps(format_obj(lda_data, lda_sim_clusters)))

In [170]:
lda_mallet_sim = corpus_similarity(mallet_doc_topics, num_topics)
lda_mallet_sim_clusters, i = get_cluster(mallet_doc_topics, num_topics, cat_names, 10, method='birch')

lda_mallet_sim_df = pd.DataFrame(lda_mallet_sim)
lda_mallet_sim_df.index = cat_names
lda_mallet_sim_df.columns = cat_names
# lda_data = top_n(lda_sim_df, 50)
lda_data = slice_df_by_name(lda_mallet_sim_df, selected_names)

with open("display/lda_mallet_output.json", "w") as f:
    f.write(json.dumps(format_obj(lda_data, lda_mallet_sim_clusters)))

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, tfidf_corpus, dictionary)
vis

In [ ]:
# largest_coherence = -1e20
# best_k = 0
# best_model = None
# for k in range(5, 150, 2):
#     model = models.LdaModel(tfidf_corpus, num_topics = k, id2word=dictionary)
#     cm = models.coherencemodel.CoherenceModel(model=model, corpus=tfidf_corpus, coherence='u_mass')
#     coherence = cm.get_coherence()
#     print("k=%d coherence=%f"%(k, coherence))
#     if (coherence > largest_coherence):
#         largest_coherence = coherence
#         best_model = model
#         best_k = k

# print("best_k:%d"%(best_k))
# for idx, topic in best_model.print_topics(-1):
#     print('Topic: {} Words: {}'.format(idx, topic))

In [ ]:
names_file = "cuisine_indices.txt"
matrix_file = "cuisine_sim_matrix.csv"

with open (names_file, 'r') as f:
    names = f.read().split("\n")

demo_data = pd.read_csv(matrix_file, header=None)
demo_data.index = names
demo_data.columns = names

with open("display/demo_output.json", "w") as f:
    f.write(json.dumps(format_obj(demo_data, np.ones(demo_data.shape[0]))))

In [ ]:
# path2reviewdump = "reviews/reviews.dat"

# with open(path2reviewdump, "r") as f:
#     reviews = f.readlines()
# review_docs = [preprocess(text) for text in tqdm(reviews)]
# review_dictionary = corpora.Dictionary(review_docs)
# print("Before prunn:%d"%(len(review_dictionary)))
# review_dictionary.filter_extremes(no_below=15, no_above = 0.5)
# print("After prunn:%d"%(len(review_dictionary)))
# review_corpus = [review_dictionary.doc2bow(doc) for doc in review_docs]

In [ ]:
# from time import time

# t0 = time()
# review_model = models.LdaModel(review_corpus, num_topics=100, id2word=review_dictionary,  eval_every=5, alpha='auto', gamma_threshold=0.01)
# print("done in %fs" % (time() - t0))

# for idx, topic in review_model.print_topics(-1):
#     print('Topic: {} Words: {}'.format(idx, topic))

In [ ]:
# def combine_topics(cat_topics):
#     topics = {}
#     for _sub_topics in cat_topics:
#         for _topic, _value in _sub_topics:
#             if _topic in topics:
#                 topics[_topic] += _value
#             else:
#                 topics[_topic] = _value
    
#     return topics

# all_topics = []
# cat_names = []
# for i in tqdm(range(0, len(cat_list))):
#     cat_names.append(cat_list[i].replace("\\", "/").split('/')[-1][:-4].replace("_"," "))
#     with open(cat_list[i]) as f:
#         cat_docs = [preprocess(text) for text in f.readlines()]
#         cat_corpus = [review_dictionary.doc2bow(doc) for doc in cat_docs]
#         cat_topics = review_model[cat_corpus]
#         all_topics.append(combine_topics(cat_topics))

In [ ]:
# lda_individual_sim = corpus_similarity([[(k, topic[k]) for k in topic] for topic in all_topics], len(review_dictionary))

# lda_individual_sim_df = pd.DataFrame(lda_individual_sim)
# lda_individual_sim_df.index = cat_names
# lda_individual_sim_df.columns = cat_names
# lda_individual_data = top_n(lda_individual_sim_df, 50)

# with open("display/lda_ind_output.json", "w") as f:
#     f.write(json.dumps(format_obj(lda_individual_data, np.ones(lda_individual_data.shape[0]))))